## Script to generate NBA game predictions for 2018-2019 season

In [2]:
# Dependencies
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os
import bs4
import splinter
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import re
import time
import datetime
from datetime import date
import sqlalchemy
from sqlalchemy import create_engine

Step 1: Machine Learning Model using SVM linear classifier

In [3]:
# Read in dataset that has regular season game data from 2012-2018
df = pd.read_csv("db/2012-18_teamBoxScore_diff_columns.csv")
df.head()

,gmDate,gmTime,seasTyp,offLNm1,offFNm1,offLNm2,offFNm2,offLNm3,offFNm3,teamAbbr,...,opptFTF,diff_teamEFG%,diff_opptEFG%,diff_teamTO%,diff_opptTO%,diff_OREB%,diff_DREB%,diff_teamFTF,diff_opptFTF,outcome
0,10/30/2012,19:00,Regular,Brothers,Tony,Smith,Michael,Workman,Haywoode,WAS,...,0.278481,-0.1000,0.1000,-7.5187,7.5187,-12.8205,-12.8205,-0.056259,0.056259,0.0
1,10/30/2012,19:00,Regular,Brothers,Tony,Smith,Michael,Workman,Haywoode,CLE,...,0.222222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/30/2012,20:00,Regular,McCutchen,Monty,Wright,Sean,Fitzgerald,Kane,BOS,...,0.405063,-0.0349,0.0349,7.5714,-7.5714,5.6006,5.6006,-0.031730,0.031730,0.0
3,10/30/2012,20:00,Regular,McCutchen,Monty,Wright,Sean,Fitzgerald,Kane,MIA,...,0.373333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10/30/2012,22:30,Regular,Foster,Scott,Zielinski,Gary,Dalen,Eric,DAL,...,0.402597,-0.0130,0.0130,-1.9419,1.9419,-10.1087,-10.1087,-0.190833,0.190833,1.0


In [4]:
df.shape

(14758, 134)

In [5]:
# drop any rows with NaN values
df = df.dropna(how='any') 

In [6]:
df.shape

(7377, 134)

In [7]:
target = df["outcome"]
target_names = ["loss", "win"]

In [8]:
# only focus on the columns for the four factors (four offense and four defense so technically 8 total)
# put that into new dataframe
df1 = df[['diff_teamEFG%','diff_opptEFG%', 'diff_teamTO%', 'diff_opptTO%', 'diff_OREB%', 'diff_DREB%', 'diff_teamFTF', 'diff_opptFTF', 'outcome']]
df1.head()

,diff_teamEFG%,diff_opptEFG%,diff_teamTO%,diff_opptTO%,diff_OREB%,diff_DREB%,diff_teamFTF,diff_opptFTF,outcome
0,-0.1000,0.1000,-7.5187,7.5187,-12.8205,-12.8205,-0.056259,0.056259,0.0
2,-0.0349,0.0349,7.5714,-7.5714,5.6006,5.6006,-0.031730,0.031730,0.0
4,-0.0130,0.0130,-1.9419,1.9419,-10.1087,-10.1087,-0.190833,0.190833,1.0
6,0.0036,-0.0036,4.6433,-4.6433,5.7300,5.7300,-0.122059,0.122059,0.0
8,0.1108,-0.1108,9.1181,-9.1181,-3.8462,-3.8462,-0.003663,0.003663,1.0


In [9]:
# new dataframe with 'outcome' dropped
data = df1.drop("outcome", axis=1)
feature_names = data.columns
data.head()

,diff_teamEFG%,diff_opptEFG%,diff_teamTO%,diff_opptTO%,diff_OREB%,diff_DREB%,diff_teamFTF,diff_opptFTF
0,-0.1000,0.1000,-7.5187,7.5187,-12.8205,-12.8205,-0.056259,0.056259
2,-0.0349,0.0349,7.5714,-7.5714,5.6006,5.6006,-0.031730,0.031730
4,-0.0130,0.0130,-1.9419,1.9419,-10.1087,-10.1087,-0.190833,0.190833
6,0.0036,-0.0036,4.6433,-4.6433,5.7300,5.7300,-0.122059,0.122059
8,0.1108,-0.1108,9.1181,-9.1181,-3.8462,-3.8462,-0.003663,0.003663


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [11]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [12]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.945


In [13]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

        loss       0.96      0.95      0.95      1094
         win       0.93      0.94      0.93       751

   micro avg       0.94      0.94      0.94      1845
   macro avg       0.94      0.94      0.94      1845
weighted avg       0.94      0.94      0.94      1845



In [14]:
# create function to make predictions
#
# returns string 'Win' if road team is predicted to win
# returns string 'Loss' if road team is predicted to lose
def predict_outcome_win_loss(road_team_abbr, road_team_stats, home_team_stats):
    road_team_array = np.array(road_team_stats)
    home_team_array = np.array(home_team_stats)
    diffs = road_team_array - home_team_array
    diffs_l = [diffs]
    prediction = model.predict(diffs_l)
    print(prediction)
    if ((prediction==1.).all()==True):
        print('Prediction is a Win for ' + str(road_team_abbr))
        outcome = 'Win'
    else:
        print('Prediction is a Loss for ' + str(road_team_abbr))
        outcome = 'Loss'
    return outcome

In [15]:
#Testing out prediction manually for sanity check purposes
#will automate this later on in this notebook

# Game DEN at SAC
den_stats = [.524, .513, 12.1, 12.2, 27.5, 77.7, .186, .191]
sac_stats = [.531, .536, 11.6, 13.7, 22.0, 74.8, .173, .209]
result = predict_outcome_win_loss('DEN', den_stats, sac_stats)
print(result)

[1.]
Prediction is a Win for DEN
Win


Step 2: Web scraping to capture updated daily stats from basketball-reference.com

In [16]:
#teams list
teams = ['ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET',
         'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN',
         'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS',
         'TOR', 'UTA', 'WAS']

In [17]:
# load web pages without loading images in selenium
from selenium import webdriver

chromeOptions = webdriver.ChromeOptions()
prefs = {'profile.managed_default_content_settings.images':2}
chromeOptions.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chrome_options=chromeOptions)

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False, options=chromeOptions)

C:\Users\zmsub\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [18]:
# loop to start the scraping of the stats
team_list = []
x = 0 
while x == 0:
    try:
        for team in teams:
            # URL
            url = r"https://www.basketball-reference.com/teams/" + team + "/2019.html"
            browser.visit(url)

            stat_list = []

            #Path to get titles of articles
            team_X = team
            oEfg_X = '//*[@id="team_misc"]/tbody/tr[1]/td[13]'
            dEfg_X = '//*[@id="team_misc"]/tbody/tr[1]/td[17]'
            oTOV_X = '//*[@id="team_misc"]/tbody/tr[1]/td[14]'
            dTOV_X = '//*[@id="team_misc"]/tbody/tr[1]/td[18]'
            oORB_X = '//*[@id="team_misc"]/tbody/tr[1]/td[15]'
            dDRB_X = '//*[@id="team_misc"]/tbody/tr[1]/td[19]'
            oFtByFga_X = '//*[@id="team_misc"]/tbody/tr[1]/td[16]'
            dFtByFga_X = '//*[@id="team_misc"]/tbody/tr[1]/td[20]'

            #Reading the results  
            abr = team_X
            stat_list.append(abr)
            oEfg = browser.find_by_xpath(oEfg_X).value
            stat_list.append(oEfg)
            dEfg = browser.find_by_xpath(dEfg_X).value
            stat_list.append(dEfg)
            oTOV = browser.find_by_xpath(oTOV_X).value
            stat_list.append(oTOV)
            dTOV = browser.find_by_xpath(dTOV_X).value
            stat_list.append(dTOV)
            oORB = browser.find_by_xpath(oORB_X).value
            stat_list.append(oORB)
            dDRB = browser.find_by_xpath(dDRB_X).value
            stat_list.append(dDRB)
            oFtByFga = browser.find_by_xpath(oFtByFga_X).value
            stat_list.append(oFtByFga)
            dFtByFga = browser.find_by_xpath(dFtByFga_X).value
            stat_list.append(dFtByFga)

            team_list.append(stat_list)
            print(team_list)
        x = 1
    except:
        print("try again")

[['ATL', '.511', '.541', '15.2', '13.3', '24.1', '76.8', '.191', '.232']]
[['ATL', '.511', '.541', '15.2', '13.3', '24.1', '76.8', '.191', '.232'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210']]
[['ATL', '.511', '.541', '15.2', '13.3', '24.1', '76.8', '.191', '.232'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210'], ['BRK', '.526', '.523', '12.5', '11.4', '24.9', '76.6', '.214', '.202']]
[['ATL', '.511', '.541', '15.2', '13.3', '24.1', '76.8', '.191', '.232'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210'], ['BRK', '.526', '.523', '12.5', '11.4', '24.9', '76.6', '.214', '.202'], ['CHO', '.519', '.529', '10.7', '12.3', '21.6', '76.7', '.205', '.185']]
[['ATL', '.511', '.541', '15.2', '13.3', '24.1', '76.8', '.191', '.232'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210'], ['BRK', '.526', '.523', '12.5', '11.4', '24.9', '76.6', '.214', '.202'], ['CHO', '.519', '.529', '10.7', '12.3', '21.6'

[['ATL', '.511', '.541', '15.2', '13.3', '24.1', '76.8', '.191', '.232'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210'], ['BRK', '.526', '.523', '12.5', '11.4', '24.9', '76.6', '.214', '.202'], ['CHO', '.519', '.529', '10.7', '12.3', '21.6', '76.7', '.205', '.185'], ['CHI', '.499', '.524', '13.4', '12.2', '17.2', '76.1', '.172', '.186'], ['CLE', '.494', '.555', '11.8', '11.0', '24.5', '76.3', '.182', '.179'], ['DAL', '.522', '.520', '13.6', '12.3', '23.5', '77.8', '.236', '.185'], ['DEN', '.524', '.516', '11.9', '12.2', '27.9', '77.7', '.186', '.190'], ['DET', '.495', '.520', '12.6', '12.6', '25.4', '79.8', '.199', '.232'], ['GSW', '.553', '.516', '12.6', '11.1', '22.8', '78.1', '.202', '.202'], ['HOU', '.535', '.529', '12.3', '12.4', '25.2', '75.4', '.219', '.218'], ['IND', '.534', '.507', '13.1', '14.3', '22.2', '77.2', '.178', '.176'], ['LAC', '.528', '.508', '12.4', '10.5', '22.7', '75.4', '.268', '.221'], ['LAL', '.533', '.504', '13.3', '11.8', '21.6', '7

[['ATL', '.511', '.541', '15.2', '13.3', '24.1', '76.8', '.191', '.232'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210'], ['BRK', '.526', '.523', '12.5', '11.4', '24.9', '76.6', '.214', '.202'], ['CHO', '.519', '.529', '10.7', '12.3', '21.6', '76.7', '.205', '.185'], ['CHI', '.499', '.524', '13.4', '12.2', '17.2', '76.1', '.172', '.186'], ['CLE', '.494', '.555', '11.8', '11.0', '24.5', '76.3', '.182', '.179'], ['DAL', '.522', '.520', '13.6', '12.3', '23.5', '77.8', '.236', '.185'], ['DEN', '.524', '.516', '11.9', '12.2', '27.9', '77.7', '.186', '.190'], ['DET', '.495', '.520', '12.6', '12.6', '25.4', '79.8', '.199', '.232'], ['GSW', '.553', '.516', '12.6', '11.1', '22.8', '78.1', '.202', '.202'], ['HOU', '.535', '.529', '12.3', '12.4', '25.2', '75.4', '.219', '.218'], ['IND', '.534', '.507', '13.1', '14.3', '22.2', '77.2', '.178', '.176'], ['LAC', '.528', '.508', '12.4', '10.5', '22.7', '75.4', '.268', '.221'], ['LAL', '.533', '.504', '13.3', '11.8', '21.6', '7

[['ATL', '.511', '.541', '15.2', '13.3', '24.1', '76.8', '.191', '.232'], ['BOS', '.531', '.505', '11.5', '13.8', '22.5', '77.8', '.175', '.210'], ['BRK', '.526', '.523', '12.5', '11.4', '24.9', '76.6', '.214', '.202'], ['CHO', '.519', '.529', '10.7', '12.3', '21.6', '76.7', '.205', '.185'], ['CHI', '.499', '.524', '13.4', '12.2', '17.2', '76.1', '.172', '.186'], ['CLE', '.494', '.555', '11.8', '11.0', '24.5', '76.3', '.182', '.179'], ['DAL', '.522', '.520', '13.6', '12.3', '23.5', '77.8', '.236', '.185'], ['DEN', '.524', '.516', '11.9', '12.2', '27.9', '77.7', '.186', '.190'], ['DET', '.495', '.520', '12.6', '12.6', '25.4', '79.8', '.199', '.232'], ['GSW', '.553', '.516', '12.6', '11.1', '22.8', '78.1', '.202', '.202'], ['HOU', '.535', '.529', '12.3', '12.4', '25.2', '75.4', '.219', '.218'], ['IND', '.534', '.507', '13.1', '14.3', '22.2', '77.2', '.178', '.176'], ['LAC', '.528', '.508', '12.4', '10.5', '22.7', '75.4', '.268', '.221'], ['LAL', '.533', '.504', '13.3', '11.8', '21.6', '7

In [19]:
# Create a dataframe that has updated stats for all of the teams
stats_df = pd.DataFrame(team_list)
stats_df.columns = ['Team_abbr','Offense_eFG','Defense_eFG','Offense_TOV','Defense_TOV','Offense_ORB','Defense_DRB','Offense_FtFga','Defense_FtFga']
stats_df

,Team_abbr,Offense_eFG,Defense_eFG,Offense_TOV,Defense_TOV,Offense_ORB,Defense_DRB,Offense_FtFga,Defense_FtFga
0,ATL,.511,.541,15.2,13.3,24.1,76.8,.191,.232
1,BOS,.531,.505,11.5,13.8,22.5,77.8,.175,.210
2,BRK,.526,.523,12.5,11.4,24.9,76.6,.214,.202
3,CHO,.519,.529,10.7,12.3,21.6,76.7,.205,.185
4,CHI,.499,.524,13.4,12.2,17.2,76.1,.172,.186
5,CLE,.494,.555,11.8,11.0,24.5,76.3,.182,.179
6,DAL,.522,.520,13.6,12.3,23.5,77.8,.236,.185
7,DEN,.524,.516,11.9,12.2,27.9,77.7,.186,.190
8,DET,.495,.520,12.6,12.6,25.4,79.8,.199,.232
9,GSW,.553,.516,12.6,11.1,22.8,78.1,.202,.202


Step 3: Update database (sqlite file) with 8 factors for each team (in 'stats' table)

In [20]:
# setup database connection
stats_engine = create_engine('sqlite:///db/schedule_abr.sqlite')
# send stats dataframe to stats table in database file
stats_df.to_sql('stats', stats_engine, if_exists='replace', index=False)

In [21]:
# test that we can query from stats table in db
stats_sql_df = pd.read_sql_query('SELECT * FROM stats',stats_engine)
stats_sql_df.head()

,Team_abbr,Offense_eFG,Defense_eFG,Offense_TOV,Defense_TOV,Offense_ORB,Defense_DRB,Offense_FtFga,Defense_FtFga
0,ATL,.511,.541,15.2,13.3,24.1,76.8,.191,.232
1,BOS,.531,.505,11.5,13.8,22.5,77.8,.175,.210
2,BRK,.526,.523,12.5,11.4,24.9,76.6,.214,.202
3,CHO,.519,.529,10.7,12.3,21.6,76.7,.205,.185
4,CHI,.499,.524,13.4,12.2,17.2,76.1,.172,.186


In [22]:
# NEED TO SET PRIMARY KEY after to_sql is done so that later sqlalchemy can interact with the stats table

import sqlite3
#connect to the database
conn = sqlite3.connect('db/schedule_abr.sqlite')
c = conn.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    ALTER TABLE stats RENAME TO old_stats;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE stats (Team_abbr text NOT NULL PRIMARY KEY,
                            Offense_eFG REAL NOT NULL,
                            Defense_eFG REAL NOT NULL,
                            Offense_TOV REAL NOT NULL,
                            Defense_TOV REAL NOT NULL,
                            Offense_ORB REAL NOT NULL,
                            Defense_DRB REAL NOT NULL,
                            Offense_FtFga REAL NOT NULL,
                            Defense_FtFga REAL NULL);

    INSERT INTO stats SELECT * FROM old_stats;

    DROP TABLE old_stats;
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
conn.close()

Step 4: Find out each game that is being played today from database

In [23]:
# setup database connection
schedule_engine = create_engine('sqlite:///db/schedule_abr.sqlite')
# test that we can query from nba_2018_2019_schedule_logo table in database file
schedule_sql_df = pd.read_sql_query('SELECT * FROM nba_2018_2019_schedule_logo',schedule_engine)
schedule_sql_df.head()

,game_id,date,time,location,home_team,road_team,home_score,road_score,result,home_team_abr,road_team_abr,road_win_prediction,home_team_logo,road_team_logo
0,1,10/16/2018,8:00 PM,TD Garden,Boston Celtics,Philadelphia 76ers,105,87,105 - 87,BOS,PHI,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
1,2,10/16/2018,10:30 PM,ORACLE Arena,Golden State Warriors,Oklahoma City Thunder,108,100,108 - 100,GSW,OKC,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
2,3,10/17/2018,7:00 PM,Spectrum Center,Charlotte Hornets,Milwaukee Bucks,112,113,112 - 113,CHO,MIL,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
3,4,10/17/2018,7:00 PM,Little Caesars Arena,Detroit Pistons,Brooklyn Nets,103,100,103 - 100,DET,BRK,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
4,5,10/17/2018,7:00 PM,Bankers Life Fieldhouse,Indiana Pacers,Memphis Grizzlies,111,83,111 - 83,IND,MEM,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...


In [24]:
# function to get today's date in same format as 'date' column of nba_2018_2019_schedule_logo table in database
# format is mm/dd/yyyy example: 1/6/2019

def get_todays_date():
    today = str(date.today())
    todays_date = date.today().strftime('%m/%d/%Y')
    print(todays_date)
    todays_date_month_first_two = todays_date[0:2]
    print(todays_date_month_first_two)
    newstr = todays_date_month_first_two
    if todays_date_month_first_two[0] == '0':
        newstr = todays_date_month_first_two.replace("0", "")
    print(newstr)
    todays_date_day_first_two = todays_date[3:5]
    print(todays_date_day_first_two)
    newstr2 = todays_date_day_first_two
    if todays_date_day_first_two[0] == '0':
        newstr2 = todays_date_day_first_two.replace("0", "")
    print(newstr2)
    newstr3 = todays_date[5:]
    final_date_string = newstr + '/' + newstr2 + newstr3
    return final_date_string

In [25]:
# test out function to get today's date
schedule_date = get_todays_date()
schedule_date

01/07/2019
01
1
07
7


'1/7/2019'

In [26]:
# function to get only today's games from schedule table into a dataframe
# this queries database

def get_todays_games(sch_date):
    cmd1 = 'SELECT * FROM nba_2018_2019_schedule_logo WHERE date='
    print(cmd1)
    cmd2 = "'"
    print(cmd2)
    cmd3 = "'"
    print(cmd3)
    cmd = cmd1 + cmd2 + schedule_date + cmd3
    print(cmd)
    schedule_today_df = pd.read_sql_query(cmd,schedule_engine)
    return schedule_today_df

In [27]:
# get only today's games into a dataframe
todays_games_df = get_todays_games(schedule_date)
todays_games_df

SELECT * FROM nba_2018_2019_schedule_logo WHERE date=
'
'
SELECT * FROM nba_2018_2019_schedule_logo WHERE date='1/7/2019'


,game_id,date,time,location,home_team,road_team,home_score,road_score,result,home_team_abr,road_team_abr,road_win_prediction,home_team_logo,road_team_logo
0,591,1/7/2019,7:00 PM,Little Caesars Arena,Detroit Pistons,San Antonio Spurs,,,,DET,SAS,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
1,592,1/7/2019,7:30 PM,TD Garden,Boston Celtics,Brooklyn Nets,,,,BOS,BRK,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
2,593,1/7/2019,8:00 PM,Toyota Center,Houston Rockets,Denver Nuggets,,,,HOU,DEN,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
3,594,1/7/2019,8:00 PM,Fiserv Forum,Milwaukee Bucks,Utah Jazz,,,,MIL,UTA,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
4,595,1/7/2019,8:00 PM,Smoothie King Center,New Orleans Pelicans,Memphis Grizzlies,,,,NOP,MEM,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
5,596,1/7/2019,8:30 PM,American Airlines Center,Dallas Mavericks,Los Angeles Lakers,,,,DAL,LAL,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
6,597,1/7/2019,10:00 PM,Moda Center,Portland Trail Blazers,New York Knicks,,,,POR,NYK,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
7,598,1/7/2019,10:00 PM,Golden 1 Center,Sacramento Kings,Orlando Magic,,,,SAC,ORL,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...


Step 5: 

a) get updated stats into arrays for road and home team's for each game being played today
b) run prediction for road team for each game being played today

In [28]:
# function to get updated stats for an individual team using stats dataframe

def capture_updated_stats(team_name, df):
    team_row = df.loc[df['Team_abbr'] == team_name]
    print('Capturing stats for ' + team_name)
    team_O_EFG = float(team_row.iloc[0][1])
    team_D_EFG = float(team_row.iloc[0][2])
    team_O_TOV = float(team_row.iloc[0][3])
    team_D_TOV = float(team_row.iloc[0][4])
    team_O_ORB = float(team_row.iloc[0][5])
    team_D_DRB = float(team_row.iloc[0][6])
    team_O_FTF = float(team_row.iloc[0][7])
    team_D_FTF = float(team_row.iloc[0][8])
    print('values after iloc and converting to float are...')
    print(team_O_EFG)
    print(team_D_EFG)
    print(team_O_TOV)
    print(team_D_TOV)
    print(team_O_ORB)
    print(team_D_DRB)
    print(team_O_FTF)
    print(team_D_FTF)
    team_stats_array = [team_O_EFG, team_D_EFG, team_O_TOV, team_D_TOV, team_O_ORB, team_D_DRB, team_O_FTF, team_D_FTF]
    return team_stats_array

In [29]:
# function to iterate through all rows of schedule dataframe
# capture stats for road team and home team
# prediction for road team is made ('Win' or 'Loss') and dataframe is modified

def prediction_iterrow(sch_df, sta_df):
    for (i, row) in sch_df.iterrows():
        home_team_abr_val = row['home_team_abr']
        road_team_abr_val = row['road_team_abr']
        road_win_prediction_val = row['road_win_prediction']
        print(home_team_abr_val, road_team_abr_val, road_win_prediction_val)
        road_team_stats = capture_updated_stats(road_team_abr_val,sta_df)
        print('Road team stats--------->')
        print(road_team_stats)
        home_team_stats = capture_updated_stats(home_team_abr_val,sta_df)
        print('Home team stats--------->')
        print(home_team_stats)
        our_prediction = predict_outcome_win_loss(road_team_abr_val, road_team_stats, home_team_stats)
        print(our_prediction)
        print('Our Prediction for ' + road_team_abr_val + ' vs. ' + home_team_abr_val + ' is: ' + our_prediction)
        sch_df.at[i,'road_win_prediction'] = our_prediction

In [30]:
# create a today's stats dataframe equal to stats_sql_df we had gotten from scraping earlier
todays_stats_df = stats_sql_df
todays_stats_df.head()

,Team_abbr,Offense_eFG,Defense_eFG,Offense_TOV,Defense_TOV,Offense_ORB,Defense_DRB,Offense_FtFga,Defense_FtFga
0,ATL,.511,.541,15.2,13.3,24.1,76.8,.191,.232
1,BOS,.531,.505,11.5,13.8,22.5,77.8,.175,.210
2,BRK,.526,.523,12.5,11.4,24.9,76.6,.214,.202
3,CHO,.519,.529,10.7,12.3,21.6,76.7,.205,.185
4,CHI,.499,.524,13.4,12.2,17.2,76.1,.172,.186


In [31]:
# show today's games dataframe again
todays_games_df.head()

,game_id,date,time,location,home_team,road_team,home_score,road_score,result,home_team_abr,road_team_abr,road_win_prediction,home_team_logo,road_team_logo
0,591,1/7/2019,7:00 PM,Little Caesars Arena,Detroit Pistons,San Antonio Spurs,,,,DET,SAS,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
1,592,1/7/2019,7:30 PM,TD Garden,Boston Celtics,Brooklyn Nets,,,,BOS,BRK,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
2,593,1/7/2019,8:00 PM,Toyota Center,Houston Rockets,Denver Nuggets,,,,HOU,DEN,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
3,594,1/7/2019,8:00 PM,Fiserv Forum,Milwaukee Bucks,Utah Jazz,,,,MIL,UTA,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
4,595,1/7/2019,8:00 PM,Smoothie King Center,New Orleans Pelicans,Memphis Grizzlies,,,,NOP,MEM,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...


In [32]:
# run function that will use todays_stats_df to modify road_win_prediction column in todays_games_df

prediction_iterrow(todays_games_df, todays_stats_df)

DET SAS Loss
Capturing stats for SAS
values after iloc and converting to float are...
0.533
0.524
11.0
11.5
21.6
78.5
0.204
0.173
Road team stats--------->
[0.533, 0.524, 11.0, 11.5, 21.6, 78.5, 0.204, 0.173]
Capturing stats for DET
values after iloc and converting to float are...
0.495
0.52
12.6
12.6
25.4
79.8
0.199
0.232
Home team stats--------->
[0.495, 0.52, 12.6, 12.6, 25.4, 79.8, 0.199, 0.232]
[1.]
Prediction is a Win for SAS
Win
Our Prediction for SAS vs. DET is: Win
BOS BRK Loss
Capturing stats for BRK
values after iloc and converting to float are...
0.526
0.523
12.5
11.4
24.9
76.6
0.214
0.202
Road team stats--------->
[0.526, 0.523, 12.5, 11.4, 24.9, 76.6, 0.214, 0.202]
Capturing stats for BOS
values after iloc and converting to float are...
0.531
0.505
11.5
13.8
22.5
77.8
0.175
0.21
Home team stats--------->
[0.531, 0.505, 11.5, 13.8, 22.5, 77.8, 0.175, 0.21]
[0.]
Prediction is a Loss for BRK
Loss
Our Prediction for BRK vs. BOS is: Loss
HOU DEN Loss
Capturing stats for DEN
va

In [33]:
# now road_win_prediction column in todays_games_df should be updated with our predictions
todays_games_df

,game_id,date,time,location,home_team,road_team,home_score,road_score,result,home_team_abr,road_team_abr,road_win_prediction,home_team_logo,road_team_logo
0,591,1/7/2019,7:00 PM,Little Caesars Arena,Detroit Pistons,San Antonio Spurs,,,,DET,SAS,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
1,592,1/7/2019,7:30 PM,TD Garden,Boston Celtics,Brooklyn Nets,,,,BOS,BRK,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
2,593,1/7/2019,8:00 PM,Toyota Center,Houston Rockets,Denver Nuggets,,,,HOU,DEN,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
3,594,1/7/2019,8:00 PM,Fiserv Forum,Milwaukee Bucks,Utah Jazz,,,,MIL,UTA,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
4,595,1/7/2019,8:00 PM,Smoothie King Center,New Orleans Pelicans,Memphis Grizzlies,,,,NOP,MEM,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
5,596,1/7/2019,8:30 PM,American Airlines Center,Dallas Mavericks,Los Angeles Lakers,,,,DAL,LAL,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
6,597,1/7/2019,10:00 PM,Moda Center,Portland Trail Blazers,New York Knicks,,,,POR,NYK,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
7,598,1/7/2019,10:00 PM,Golden 1 Center,Sacramento Kings,Orlando Magic,,,,SAC,ORL,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...


In [34]:
# update SQL database in a table called 'today_predictions' with our predictions for today's games
schedule_abr_engine = create_engine('sqlite:///db/schedule_abr.sqlite')
todays_games_df.to_sql('today_predictions', schedule_abr_engine, if_exists='replace', index=False)



In [35]:
# test that we can query from today_predictions table in db
today_predictions_sql_df = pd.read_sql_query('SELECT * FROM today_predictions',schedule_abr_engine)
today_predictions_sql_df.head()

,game_id,date,time,location,home_team,road_team,home_score,road_score,result,home_team_abr,road_team_abr,road_win_prediction,home_team_logo,road_team_logo
0,591,1/7/2019,7:00 PM,Little Caesars Arena,Detroit Pistons,San Antonio Spurs,,,,DET,SAS,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
1,592,1/7/2019,7:30 PM,TD Garden,Boston Celtics,Brooklyn Nets,,,,BOS,BRK,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
2,593,1/7/2019,8:00 PM,Toyota Center,Houston Rockets,Denver Nuggets,,,,HOU,DEN,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
3,594,1/7/2019,8:00 PM,Fiserv Forum,Milwaukee Bucks,Utah Jazz,,,,MIL,UTA,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
4,595,1/7/2019,8:00 PM,Smoothie King Center,New Orleans Pelicans,Memphis Grizzlies,,,,NOP,MEM,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...


In [36]:
# NEED TO SET PRIMARY KEY after to_sql is done so that later sqlalchemy can interact with the today_predictions table

import sqlite3
#connect to the database
conn = sqlite3.connect('db/schedule_abr.sqlite')
c = conn.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    ALTER TABLE today_predictions RENAME TO old_table;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE today_predictions (game_id INTEGER NOT NULL PRIMARY KEY,
                            date text NOT NULL,
                            time text NOT NULL,
                            location text NOT NULL,
                            home_team text NOT NULL,
                            road_team text NOT NULL,
                            home_score INTEGER NOT NULL,
                            road_score INTEGER NOT NULL,
                            result text NOT NULL,
                            home_team_abr text NOT NULL,
                            road_team_abr text NOT NULL,
                            road_win_prediction text NOT NULL,
                            home_team_logo text NOT NULL,
                            road_team_logo text NOT NULL);

    INSERT INTO today_predictions SELECT * FROM old_table;

    DROP TABLE old_table;
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
conn.close()

Step 6: 

a) get updated stats into arrays for road and home team's for each game in 2019 (rest of season)
b) run prediction for road team for each game being played in 2019 (rest of season)

In [37]:
# create a current stats dataframe equal to stats_sql_df we had gotten from scraping earlier
current_stats_df = stats_sql_df
current_stats_df.head()

,Team_abbr,Offense_eFG,Defense_eFG,Offense_TOV,Defense_TOV,Offense_ORB,Defense_DRB,Offense_FtFga,Defense_FtFga
0,ATL,.511,.541,15.2,13.3,24.1,76.8,.191,.232
1,BOS,.531,.505,11.5,13.8,22.5,77.8,.175,.210
2,BRK,.526,.523,12.5,11.4,24.9,76.6,.214,.202
3,CHO,.519,.529,10.7,12.3,21.6,76.7,.205,.185
4,CHI,.499,.524,13.4,12.2,17.2,76.1,.172,.186


In [56]:
# function to get a year's games from schedule table into a dataframe
# this queries database

def get_year_games(year):
    cmd1 = 'SELECT game_id, date, home_team, road_team, home_team_abr, road_team_abr, road_win_prediction, home_team_logo, road_team_logo FROM nba_2018_2019_schedule_logo WHERE substr(date, 5, 4)='
    print(cmd1)
    cmd2 = "'"
    print(cmd2)
    cmd3 = "'"
    print(cmd3)
    cmd4 = 'or substr(date, 6, 4)='
    print(cmd4)
    cmd5 = "'"
    print(cmd2)
    cmd6 = "'"
    print(cmd3)
    cmd = cmd1 + cmd2 + year + cmd3 + cmd4 + cmd5 + year + cmd6
    print(cmd)
    schedule_year_df = pd.read_sql_query(cmd,schedule_engine)
    return schedule_year_df

In [57]:
# get 2019's games into a dataframe
season_year = '2019'
year_games_df = get_year_games(season_year)
year_games_df.head()

SELECT game_id, date, home_team, road_team, home_team_abr, road_team_abr, road_win_prediction, home_team_logo, road_team_logo FROM nba_2018_2019_schedule_logo WHERE substr(date, 5, 4)=
'
'
or substr(date, 6, 4)=
'
'
SELECT game_id, date, home_team, road_team, home_team_abr, road_team_abr, road_win_prediction, home_team_logo, road_team_logo FROM nba_2018_2019_schedule_logo WHERE substr(date, 5, 4)='2019'or substr(date, 6, 4)='2019'


,game_id,date,home_team,road_team,home_team_abr,road_team_abr,road_win_prediction,home_team_logo,road_team_logo
0,549,1/1/2019,Toronto Raptors,Utah Jazz,TOR,UTA,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
1,550,1/1/2019,Milwaukee Bucks,Detroit Pistons,MIL,DET,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
2,551,1/1/2019,Denver Nuggets,New York Knicks,DEN,NYK,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
3,552,1/1/2019,Sacramento Kings,Portland Trail Blazers,SAC,POR,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
4,553,1/1/2019,LA Clippers,Philadelphia 76ers,LAC,PHI,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...


In [58]:
# run function that will use current_stats_df to modify road_win_prediction column in year_games_df

prediction_iterrow(year_games_df, current_stats_df)

TOR UTA Loss
Capturing stats for UTA
values after iloc and converting to float are...
0.529
0.519
13.6
13.4
21.9
80.0
0.217
0.195
Road team stats--------->
[0.529, 0.519, 13.6, 13.4, 21.9, 80.0, 0.217, 0.195]
Capturing stats for TOR
values after iloc and converting to float are...
0.539
0.509
11.8
12.5
23.0
76.2
0.194
0.185
Home team stats--------->
[0.539, 0.509, 11.8, 12.5, 23.0, 76.2, 0.194, 0.185]
[0.]
Prediction is a Loss for UTA
Loss
Our Prediction for UTA vs. TOR is: Loss
MIL DET Loss
Capturing stats for DET
values after iloc and converting to float are...
0.495
0.52
12.6
12.6
25.4
79.8
0.199
0.232
Road team stats--------->
[0.495, 0.52, 12.6, 12.6, 25.4, 79.8, 0.199, 0.232]
Capturing stats for MIL
values after iloc and converting to float are...
0.554
0.502
12.3
11.6
21.4
81.1
0.194
0.171
Home team stats--------->
[0.554, 0.502, 12.3, 11.6, 21.4, 81.1, 0.194, 0.171]
[0.]
Prediction is a Loss for DET
Loss
Our Prediction for DET vs. MIL is: Loss
DEN NYK Loss
Capturing stats for N

74.2
0.182
0.235
Home team stats--------->
[0.517, 0.543, 14.0, 13.4, 20.3, 74.2, 0.182, 0.235]
[1.]
Prediction is a Win for LAC
Win
Our Prediction for LAC vs. PHO is: Win
LAL NYK Loss
Capturing stats for NYK
values after iloc and converting to float are...
0.488
0.544
11.5
11.9
24.0
75.8
0.204
0.202
Road team stats--------->
[0.488, 0.544, 11.5, 11.9, 24.0, 75.8, 0.204, 0.202]
Capturing stats for LAL
values after iloc and converting to float are...
0.533
0.504
13.3
11.8
21.6
76.5
0.179
0.184
Home team stats--------->
[0.533, 0.504, 13.3, 11.8, 21.6, 76.5, 0.179, 0.184]
[0.]
Prediction is a Loss for NYK
Loss
Our Prediction for NYK vs. LAL is: Loss
POR OKC Loss
Capturing stats for OKC
values after iloc and converting to float are...
0.505
0.506
12.1
15.1
26.6
78.1
0.188
0.196
Road team stats--------->
[0.505, 0.506, 12.1, 15.1, 26.6, 78.1, 0.188, 0.196]
Capturing stats for POR
values after iloc and converting to float are...
0.515
0.517
12.5
10.7
25.2
78.2
0.208
0.179
Home team stats---

Loss
Our Prediction for NYK vs. GSW is: Loss
LAC CHO Loss
Capturing stats for CHO
values after iloc and converting to float are...
0.519
0.529
10.7
12.3
21.6
76.7
0.205
0.185
Road team stats--------->
[0.519, 0.529, 10.7, 12.3, 21.6, 76.7, 0.205, 0.185]
Capturing stats for LAC
values after iloc and converting to float are...
0.528
0.508
12.4
10.5
22.7
75.4
0.268
0.221
Home team stats--------->
[0.528, 0.508, 12.4, 10.5, 22.7, 75.4, 0.268, 0.221]
[0.]
Prediction is a Loss for CHO
Loss
Our Prediction for CHO vs. LAC is: Loss
BOS IND Loss
Capturing stats for IND
values after iloc and converting to float are...
0.534
0.507
13.1
14.3
22.2
77.2
0.178
0.176
Road team stats--------->
[0.534, 0.507, 13.1, 14.3, 22.2, 77.2, 0.178, 0.176]
Capturing stats for BOS
values after iloc and converting to float are...
0.531
0.505
11.5
13.8
22.5
77.8
0.175
0.21
Home team stats--------->
[0.531, 0.505, 11.5, 13.8, 22.5, 77.8, 0.175, 0.21]
[0.]
Prediction is a Loss for IND
Loss
Our Prediction for IND vs. BO

0.53
0.503
13.1
11.0
23.3
78.8
0.246
0.214
Road team stats--------->
[0.53, 0.503, 13.1, 11.0, 23.3, 78.8, 0.246, 0.214]
Capturing stats for NYK
values after iloc and converting to float are...
0.488
0.544
11.5
11.9
24.0
75.8
0.204
0.202
Home team stats--------->
[0.488, 0.544, 11.5, 11.9, 24.0, 75.8, 0.204, 0.202]
[1.]
Prediction is a Win for PHI
Win
Our Prediction for PHI vs. NYK is: Win
WAS TOR Loss
Capturing stats for TOR
values after iloc and converting to float are...
0.539
0.509
11.8
12.5
23.0
76.2
0.194
0.185
Road team stats--------->
[0.539, 0.509, 11.8, 12.5, 23.0, 76.2, 0.194, 0.185]
Capturing stats for WAS
values after iloc and converting to float are...
0.523
0.539
11.9
13.7
20.2
72.8
0.208
0.215
Home team stats--------->
[0.523, 0.539, 11.9, 13.7, 20.2, 72.8, 0.208, 0.215]
[1.]
Prediction is a Win for TOR
Win
Our Prediction for TOR vs. WAS is: Win
ATL MIL Loss
Capturing stats for MIL
values after iloc and converting to float are...
0.554
0.502
12.3
11.6
21.4
81.1
0.194
0.

0.184
Road team stats--------->
[0.533, 0.504, 13.3, 11.8, 21.6, 76.5, 0.179, 0.184]
Capturing stats for OKC
values after iloc and converting to float are...
0.505
0.506
12.1
15.1
26.6
78.1
0.188
0.196
Home team stats--------->
[0.505, 0.506, 12.1, 15.1, 26.6, 78.1, 0.188, 0.196]
[0.]
Prediction is a Loss for LAL
Loss
Our Prediction for LAL vs. OKC is: Loss
BOS MEM Loss
Capturing stats for MEM
values after iloc and converting to float are...
0.511
0.514
12.4
14.4
17.8
77.5
0.206
0.237
Road team stats--------->
[0.511, 0.514, 12.4, 14.4, 17.8, 77.5, 0.206, 0.237]
Capturing stats for BOS
values after iloc and converting to float are...
0.531
0.505
11.5
13.8
22.5
77.8
0.175
0.21
Home team stats--------->
[0.531, 0.505, 11.5, 13.8, 22.5, 77.8, 0.175, 0.21]
[0.]
Prediction is a Loss for MEM
Loss
Our Prediction for MEM vs. BOS is: Loss
DET MIA Loss
Capturing stats for MIA
values after iloc and converting to float are...
0.503
0.502
12.6
12.3
26.4
77.0
0.188
0.214
Road team stats--------->
[0

...

78.8
0.246
0.214
Road team stats--------->
[0.53, 0.503, 13.1, 11.0, 23.3, 78.8, 0.246, 0.214]
Capturing stats for ATL
values after iloc and converting to float are...
0.511
0.541
15.2
13.3
24.1
76.8
0.191
0.232
Home team stats--------->
[0.511, 0.541, 15.2, 13.3, 24.1, 76.8, 0.191, 0.232]
[1.]
Prediction is a Win for PHI
Win
Our Prediction for PHI vs. ATL is: Win
CHI UTA Loss
Capturing stats for UTA
values after iloc and converting to float are...
0.529
0.519
13.6
13.4
21.9
80.0
0.217
0.195
Road team stats--------->
[0.529, 0.519, 13.6, 13.4, 21.9, 80.0, 0.217, 0.195]
Capturing stats for CHI
values after iloc and converting to float are...
0.499
0.524
13.4
12.2
17.2
76.1
0.172
0.186
Home team stats--------->
[0.499, 0.524, 13.4, 12.2, 17.2, 76.1, 0.172, 0.186]
[1.]
Prediction is a Win for UTA
Win
Our Prediction for UTA vs. CHI is: Win
MEM MIN Loss
Capturing stats for MIN
values after iloc and converting to float are...
0.508
0.526
11.2
13.3
24.4
74.3
0.213
0.183
Road team stats-------

0.529
0.519
13.6
13.4
21.9
80.0
0.217
0.195
Home team stats--------->
[0.529, 0.519, 13.6, 13.4, 21.9, 80.0, 0.217, 0.195]
[0.]
Prediction is a Loss for LAL
Loss
Our Prediction for LAL vs. UTA is: Loss
DET ORL Loss
Capturing stats for ORL
values after iloc and converting to float are...
0.507
0.52
12.0
12.0
20.1
77.7
0.164
0.197
Road team stats--------->
[0.507, 0.52, 12.0, 12.0, 20.1, 77.7, 0.164, 0.197]
Capturing stats for DET
values after iloc and converting to float are...
0.495
0.52
12.6
12.6
25.4
79.8
0.199
0.232
Home team stats--------->
[0.495, 0.52, 12.6, 12.6, 25.4, 79.8, 0.199, 0.232]
[0.]
Prediction is a Loss for ORL
Loss
Our Prediction for ORL vs. DET is: Loss
PHI BRK Loss
Capturing stats for BRK
values after iloc and converting to float are...
0.526
0.523
12.5
11.4
24.9
76.6
0.214
0.202
Road team stats--------->
[0.526, 0.523, 12.5, 11.4, 24.9, 76.6, 0.214, 0.202]
Capturing stats for PHI
values after iloc and converting to float are...
0.53
0.503
13.1
11.0
23.3
78.8
0.246

0.21
Home team stats--------->
[0.531, 0.505, 11.5, 13.8, 22.5, 77.8, 0.175, 0.21]
[0.]
Prediction is a Loss for MIA
Loss
Our Prediction for MIA vs. BOS is: Loss
BRK MIL Loss
Capturing stats for MIL
values after iloc and converting to float are...
0.554
0.502
12.3
11.6
21.4
81.1
0.194
0.171
Road team stats--------->
[0.554, 0.502, 12.3, 11.6, 21.4, 81.1, 0.194, 0.171]
Capturing stats for BRK
values after iloc and converting to float are...
0.526
0.523
12.5
11.4
24.9
76.6
0.214
0.202
Home team stats--------->
[0.526, 0.523, 12.5, 11.4, 24.9, 76.6, 0.214, 0.202]
[1.]
Prediction is a Win for MIL
Win
Our Prediction for MIL vs. BRK is: Win
NYK CHI Loss
Capturing stats for CHI
values after iloc and converting to float are...
0.499
0.524
13.4
12.2
17.2
76.1
0.172
0.186
Road team stats--------->
[0.499, 0.524, 13.4, 12.2, 17.2, 76.1, 0.172, 0.186]
Capturing stats for NYK
values after iloc and converting to float are...
0.488
0.544
11.5
11.9
24.0
75.8
0.204
0.202
Home team stats--------->
[0.48

OKC DET Loss
Capturing stats for DET
values after iloc and converting to float are...
0.495
0.52
12.6
12.6
25.4
79.8
0.199
0.232
Road team stats--------->
[0.495, 0.52, 12.6, 12.6, 25.4, 79.8, 0.199, 0.232]
Capturing stats for OKC
values after iloc and converting to float are...
0.505
0.506
12.1
15.1
26.6
78.1
0.188
0.196
Home team stats--------->
[0.505, 0.506, 12.1, 15.1, 26.6, 78.1, 0.188, 0.196]
[0.]
Prediction is a Loss for DET
Loss
Our Prediction for DET vs. OKC is: Loss
DAL MEM Loss
Capturing stats for MEM
values after iloc and converting to float are...
0.511
0.514
12.4
14.4
17.8
77.5
0.206
0.237
Road team stats--------->
[0.511, 0.514, 12.4, 14.4, 17.8, 77.5, 0.206, 0.237]
Capturing stats for DAL
values after iloc and converting to float are...
0.522
0.52
13.6
12.3
23.5
77.8
0.236
0.185
Home team stats--------->
[0.522, 0.52, 13.6, 12.3, 23.5, 77.8, 0.236, 0.185]
[0.]
Prediction is a Loss for MEM
Loss
Our Prediction for MEM vs. DAL is: Loss
UTA SAC Loss
Capturing stats for SAC

12.3
12.4
25.2
75.4
0.219
0.218
Road team stats--------->
[0.535, 0.529, 12.3, 12.4, 25.2, 75.4, 0.219, 0.218]
Capturing stats for OKC
values after iloc and converting to float are...
0.505
0.506
12.1
15.1
26.6
78.1
0.188
0.196
Home team stats--------->
[0.505, 0.506, 12.1, 15.1, 26.6, 78.1, 0.188, 0.196]
[0.]
Prediction is a Loss for HOU
Loss
Our Prediction for HOU vs. OKC is: Loss
LAL POR Loss
Capturing stats for POR
values after iloc and converting to float are...
0.515
0.517
12.5
10.7
25.2
78.2
0.208
0.179
Road team stats--------->
[0.515, 0.517, 12.5, 10.7, 25.2, 78.2, 0.208, 0.179]
Capturing stats for LAL
values after iloc and converting to float are...
0.533
0.504
13.3
11.8
21.6
76.5
0.179
0.184
Home team stats--------->
[0.533, 0.504, 13.3, 11.8, 21.6, 76.5, 0.179, 0.184]
[0.]
Prediction is a Loss for POR
Loss
Our Prediction for POR vs. LAL is: Loss
ATL IND Loss
Capturing stats for IND
values after iloc and converting to float are...
0.534
0.507
13.1
14.3
22.2
77.2
0.178
0.176


In [59]:
# now road_win_prediction column in year_games_df should be updated with our predictions
year_games_df.head

<bound method NDFrame.head of      game_id       date               home_team               road_team  \
0        549   1/1/2019         Toronto Raptors               Utah Jazz   
1        550   1/1/2019         Milwaukee Bucks         Detroit Pistons   
2        551   1/1/2019          Denver Nuggets         New York Knicks   
3        552   1/1/2019        Sacramento Kings  Portland Trail Blazers   
4        553   1/1/2019             LA Clippers      Philadelphia 76ers   
5        554   1/2/2019       Charlotte Hornets        Dallas Mavericks   
6        555   1/2/2019     Cleveland Cavaliers              Miami Heat   
7        556   1/2/2019      Washington Wizards           Atlanta Hawks   
8        557   1/2/2019           Brooklyn Nets    New Orleans Pelicans   
9        558   1/2/2019          Boston Celtics  Minnesota Timberwolves   
10       559   1/2/2019           Chicago Bulls           Orlando Magic   
11       560   1/2/2019       Memphis Grizzlies         Detroit Piston

In [62]:
# update SQL database in a table called 'year_predictions' with our predictions for the rest of season (year) games
year_schedule_abr_engine = create_engine('sqlite:///db/schedule_abr.sqlite')
year_games_df.to_sql('year_predictions', year_schedule_abr_engine, if_exists='replace', chunksize=75, index=False)


In [63]:
# test that we can query from year_predictions table in db
year_predictions_sql_df = pd.read_sql_query('SELECT * FROM year_predictions',schedule_abr_engine)
year_predictions_sql_df.head()

,game_id,date,home_team,road_team,home_team_abr,road_team_abr,road_win_prediction,home_team_logo,road_team_logo
0,549,1/1/2019,Toronto Raptors,Utah Jazz,TOR,UTA,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
1,550,1/1/2019,Milwaukee Bucks,Detroit Pistons,MIL,DET,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
2,551,1/1/2019,Denver Nuggets,New York Knicks,DEN,NYK,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
3,552,1/1/2019,Sacramento Kings,Portland Trail Blazers,SAC,POR,Loss,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...
4,553,1/1/2019,LA Clippers,Philadelphia 76ers,LAC,PHI,Win,https://www.nba.com/assets/logos/teams/primary...,https://www.nba.com/assets/logos/teams/primary...


In [65]:
# NEED TO SET PRIMARY KEY after to_sql is done so that later sqlalchemy can interact with the today_predictions table

import sqlite3
#connect to the database
conn = sqlite3.connect('db/schedule_abr.sqlite')
c = conn.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    ALTER TABLE year_predictions RENAME TO old_year_table;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE year_predictions (game_id INTEGER NOT NULL PRIMARY KEY,
                            date text NOT NULL,
                            home_team text NOT NULL,
                            road_team text NOT NULL,
                            home_team_abr text NOT NULL,
                            road_team_abr text NOT NULL,
                            road_win_prediction text NOT NULL,
                            home_team_logo text NOT NULL,
                            road_team_logo text NOT NULL);

    INSERT INTO year_predictions SELECT * FROM old_year_table;

    DROP TABLE old_year_table;
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
conn.close()